In [1]:
from zad11 import bench, connect, load_table, process

In [2]:
spark = connect()

Connecting to Spark...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/09 07:10:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/09 07:10:12 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
daily_weather_2017 = load_table(spark, "/input/daily_weather_2017.csv")

Loading '/input/daily_weather_2017.csv' into table 'daily_weather_2017'...


In [4]:
cities = load_table(spark, "/input/cities.csv")

Loading '/input/cities.csv' into table 'cities'...


In [5]:
from pyspark.sql.functions import to_date, col, avg, coalesce, lit

result_df = (
    (
        (
            daily_weather_2017.filter(
                (col("date") >= "2017-01-01") & (col("date") <= "2021-12-31")
            ).select(
                col("station_id"),
                to_date(col("date")).alias("date"),
                col("avg_temp_c").alias("temperature_c"),
                col("precipitation_mm"),
            )
        )
        .join(cities, on="station_id")
        .groupBy("country", "date")
        .agg(
            avg("temperature_c").alias("temperature_c"),
            coalesce(avg("precipitation_mm"), lit(0)).alias("precipitation_mm"),
        )
    )
    .filter(col("temperature_c").isNotNull())
    .orderBy("country", "date")
)


with bench():
    result_df.show()

+-----------+----------+--------------------+----------------+
|    country|      date|       temperature_c|precipitation_mm|
+-----------+----------+--------------------+----------------+
|Afghanistan|2017-01-01|  5.3166666666666655|             0.0|
|Afghanistan|2017-01-02|   5.016666666666667|             2.0|
|Afghanistan|2017-01-03|  3.0666666666666664|          10.725|
|Afghanistan|2017-01-04|                2.65|           109.0|
|Afghanistan|2017-01-05|  1.9333333333333333|           29.95|
|Afghanistan|2017-01-06|  0.9833333333333331|            7.25|
|Afghanistan|2017-01-07|  0.2833333333333334|            33.0|
|Afghanistan|2017-01-08|-0.07999999999999999|            18.0|
|Afghanistan|2017-01-09|-0.21666666666666665|             0.0|
|Afghanistan|2017-01-10|   0.866666666666667|             0.0|
|Afghanistan|2017-01-11| 0.44999999999999973|             0.0|
|Afghanistan|2017-01-12| -0.6166666666666668|             0.0|
|Afghanistan|2017-01-13| -0.8399999999999996|          

In [6]:
spark.stop()